In [ ]:
import asyncio
import schedule
import time
from datetime import datetime, timedelta, timezone
import pytz
import requests
from telethon.sync import TelegramClient
from telethon.tl.functions.messages import GetHistoryRequest
from openai import OpenAI
import os

# === API ===
DEEPSEEK_API_KEY = "your_api"
TELEGRAM_BOT_TOKEN = "your_bot_token"
TELEGRAM_CHANNEL = "@channel"
API_ID = api_id
API_HASH = 'your_api_hash'

CHANNELS = [
    "doubletop",
    "investcoin_free",
    "incrypted",
    "Maykanlnvest",
    "ctpc_team",
    "micro_vyshnya",
    "cryptokitta",
    "CRYPTUS_MEDIA",
    "cryptokucherov",
    "procent_crypto",
    "whitelist1",
    "crypto_miracle01",
    "forklog",
    "ProMintChannel",
    "kryptorom",
    "bloggerberkut",
    "ZAVODCHENELL",
    "cryptoonick",
    "one_million_airdrop",
    "Hunt_for_Drops",
    "SaversCrypto",
    "crypto_shine01",
    "cryptoforto"
]

# === Fetch cryptocurrency data from CoinGecko ===
def get_crypto_data():
    url = "https://api.coingecko.com/api/v3/global"
    global_data = requests.get(url).json()
    btc_dominance = global_data["data"]["market_cap_percentage"]["btc"]

    price_url = "https://api.coingecko.com/api/v3/simple/price"
    params = {
        "ids": "bitcoin,ethereum",
        "vs_currencies": "usd"
    }
    prices = requests.get(price_url, params=params).json()

    btc_price = prices["bitcoin"]["usd"]
    eth_price = prices["ethereum"]["usd"]

    return round(btc_price, 2), round(eth_price, 2), round(btc_dominance, 2)

# === Clean text by removing '#' and '*' characters ===
def clean_text(text):
    return text.replace("#", "").replace("*", "")

# === Send message to Telegram channel ===
def send_to_telegram(text):
    url = f"https://api.telegram.org/bot{TELEGRAM_BOT_TOKEN}/sendMessage"
    payload = {
        "chat_id": TELEGRAM_CHANNEL,
        "text": text
    }
    r = requests.post(url, data=payload)
    if r.status_code == 200:
        print("✅ Post sent to Telegram!")
    else:
        print("⚠️ Sending error:", r.text)

# === Fetch recent messages from a Telegram channel within last 24 hours ===
async def fetch_recent_messages(channel_username, limit=100):
    try:
        entity = await client.get_entity(channel_username)
        history = await client(GetHistoryRequest(
            peer=entity,
            limit=limit,
            offset_date=None,
            offset_id=0,
            max_id=0,
            min_id=0,
            add_offset=0,
            hash=0
        ))

        messages = []
        now = datetime.now(timezone.utc)
        one_day_ago = now - timedelta(days=1)

        for msg in history.messages:
            if msg.message and msg.date >= one_day_ago:
                messages.append(msg.message.strip())

        return messages
    except Exception as e:
        print(f"⚠️ Error with @{channel_username}: {e}")
        return []
    
# === Query DeepSeek API with a prompt ===
def ask_deepseek(prompt):
    url = "https://api.deepseek.com/v1/chat/completions"  # Current API URL
    headers = {
        "Authorization": f"Bearer {DEEPSEEK_API_KEY}",
        "Content-Type": "application/json"
    }
    json_data = {
        "model": "deepseek-chat",  # or "deepseek/deepseek-chat" depending on your setup
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 1000
    }

    response = requests.post(url, headers=headers, json=json_data)
    response.raise_for_status()
    data = response.json()

    # Extract the generated text directly from the response
    return data["choices"][0]["message"]["content"]


# === Main task: fetch posts, summarize, and send to Telegram ===
async def main_job():
    print(f"🚀 Starting task at {datetime.now(pytz.timezone('Europe/Moscow')).strftime('%Y-%m-%d %H:%M:%S')} (Moscow Time)")

    global client
    client = TelegramClient("session", API_ID, API_HASH)
    await client.start()

    all_posts = []
    for channel in CHANNELS:
        print(f"🔍 Fetching posts from @{channel}...")
        msgs = await fetch_recent_messages(channel)
        for msg in msgs:
            all_posts.append({"Channel": f"@{channel}", "Text": msg})

    await client.disconnect()

    if not all_posts:
        print("⚠️ No new posts found in the last 24 hours. Skipping send.")
        return

    all_posts_text = "\n".join([f"{m['Text']} ({m['Channel']})" for m in all_posts])
    btc_price, eth_price, btc_dominance = get_crypto_data()

    prompt = f"""
Write a Telegram post including the following:
BTC price = {btc_price}
ETH price = {eth_price}
BTC dominance = {btc_dominance}

Airdrop news: find and list up to 5 news items related to airdrops, retro hunting, claims, or similar keywords.
Crypto news: a brief summary of up to 5 recent cryptocurrency news items.

After each news item, mention the channel where the information was found.
Do not use '#' or '*', replace them with spaces.
Do not add unnecessary phrases at the end.

{all_posts_text}
"""

    response = ask_deepseek(prompt)
    
    post = clean_text(response)
    print("=== Generated post ===\n", post)
    send_to_telegram(post)
    

await main_job()